# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [19]:
# # insider db
# wb = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# fold path

In [3]:
drive = 'D'

In [4]:
raw_fold = f'{drive}:/data/big2/BD1/raw'
ods_fold = f'{drive}:/data/big2/ODS'
sample_fold = f'{drive}:/data/big2/BD1/sample'
analysis_fold = f'{drive}:/data/big2/BD1/an'
df_fold = f'{drive}:/data/big2/BD1/df'

# Load
- 등록&제원 -> result 4등급(연료 : 경유, 휘발유, LPG)

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [4]:
# 8.6s
car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")
# car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD FROM STD_CEG_CAR_MIG;")

In [5]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898814 entries, 0 to 1898813
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   VIN             object 
 1   BSPL_STDG_CD    int64  
 2   VHCL_ERSR_YN    object 
 3   MANP_MNG_NO     object 
 4   YRIDNW          int64  
 5   VHCTY_CD        object 
 6   PURPS_CD2       object 
 7   FRST_REG_YMD    int64  
 8   VHCL_FBCTN_YMD  float64
 9   VHCL_MNG_NO     object 
dtypes: float64(1), int64(3), object(6)
memory usage: 144.9+ MB


In [6]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호',
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
    'VHCL_MNG_NO':'차량관리번호'
}

In [39]:
carr = car.rename(columns=car_ch_col)

In [40]:
carr.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [9]:
# 3.8s
src = wb.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2 FROM STD_CEG_CAR_SRC_MIG;")

In [10]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
}

In [11]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형'], dtype='object')

## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [12]:
# 3.0s
att = wb.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010946 entries, 0 to 1010945
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   VIN           1010946 non-null  object
 1   RDCDVC_SE_CD  1010946 non-null  object
dtypes: object(2)
memory usage: 15.4+ MB


In [13]:
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분', 
}

In [14]:
attr = att.rename(columns=att_ch_col)
attr.columns

Index(['차대번호', '저감장치구분'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [15]:
# 1.3s
code = wb.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_NM  46333 non-null  object
 2   STDG_SGG_NM   46310 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [16]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [17]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [18]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## 등록이력(CEG_CAR_HISTORY_MIG)

In [20]:
# 1m 8.2s
his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
his.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50044247 entries, 0 to 50044246
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   VHCL_ERSR_YN  object 
 1   CHNG_DE       float64
 2   VHMNO         object 
dtypes: float64(1), object(2)
memory usage: 1.1+ GB


In [21]:
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}

In [22]:
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

# 전처리

## 중복 차대번호 제거

In [41]:
carr.shape, len(carr['차대번호'].unique())

((1898814, 10), 1898763)

In [42]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(1898763, 10)

## 차종 코드 변환

In [43]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '승합', '승용', '특수', nan], dtype=object)

## 용도 코드 변환

In [44]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [45]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1515737, 10)

## 연료 코드 변환

In [28]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', '등유', '태양열', 'CNG 하이브리드', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [29]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [30]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1515737, 12)

## 저감장치구분 코드 변환

In [31]:
# 4m 14.3s
attr['저감장치구분'].unique()

array(['A1001', 'A1003', 'A1005', 'A1002', 'A1006', 'A1004', 'A1007'],
      dtype=object)

In [32]:
# 27.5s
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '2종', '엔진교체', '1종+SCR', '삼원촉매'], dtype=object)

## 저감장치 부착 유무

In [33]:
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'

In [34]:
attr['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    531833
유      479113
Name: count, dtype: int64

## 등록&제원&저감이력 병합

In [35]:
# 1.7s
csa = cs.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
csa.shape

(1515737, 13)

## 지역정보 병합

In [52]:
df = csa.merge(coder, on='법정동코드', how='left')

### 매칭 안되는 지역 처리

In [54]:
# 경기도 양주시
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시']

# 전처리

## 연료 확인

In [55]:
df['연료'].value_counts(dropna=False)

연료
경유             1156240
휘발유             333206
LPG(액화석유가스)      26179
NaN                 71
기타연료                39
CNG(압축천연가스)          1
알코올                  1
Name: count, dtype: int64

## 연료 컬럼 추가

In [56]:
df1 = df.copy()

In [57]:
df1.loc[df1['연료'] == '경유', 'fuel'] = '경유'
df1.loc[(df1['연료'] == '휘발유') | (df1['연료'] == 'LPG(액화석유가스)'), 'fuel'] = '휘발유_가스'

# 분석

## EG 분류

In [63]:
grade_list = []
for f, y, cy, e in tqdm(df1[['fuel', '제작일자', '차량연식', 'DPF_YN']].values):
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

100%|██████████| 1515737/1515737 [00:03<00:00, 464793.74it/s]


1515737

In [64]:
df1['Grade'] = grade_list

In [75]:
df2 = df1[[
    '차대번호', 
    '제원관리번호',
    '차종', 
    '용도', 
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도', 
    '시군구', 
    'DPF_YN',
    'Grade',
    ]]
df2.columns

Index(['차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드', '시도',
       '시군구', 'DPF_YN', 'Grade'],
      dtype='object')

In [76]:
today_date = datetime.today().strftime("%Y%m")
today_date

'202306'

In [77]:
df2['기준연월'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_10156\822676589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['기준연월'] = today_date


In [78]:
df2.columns

Index(['차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드', '시도',
       '시군구', 'DPF_YN', 'Grade', '기준연월'],
      dtype='object')

In [79]:
df3 = df2[[
    '기준연월', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도', 
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구', 
    'DPF_YN', 
    'Grade', 
    ]]
df3.columns

Index(['기준연월', '차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드',
       '시도', '시군구', 'DPF_YN', 'Grade'],
      dtype='object')

In [80]:
ch_col_dict = {
                '기준연월':'CRTR_YM',
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO',
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2',
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM',
                '시군구':'SGG_NM',
                'DPF_YN':'DPF_MNTNG_YN',
                'Grade':'GRD4_MLSFC', 
                }

In [81]:
STD_BD_GRD4_MLSFC_RSLT = df3.rename(columns=ch_col_dict)
STD_BD_GRD4_MLSFC_RSLT.columns

Index(['CRTR_YM', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2', 'YRIDNW',
       'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'DPF_MNTNG_YN',
       'GRD4_MLSFC'],
      dtype='object')

## [출력] STD_BD_GRD4_MLSFC_RSLT

In [82]:
expdf = STD_BD_GRD4_MLSFC_RSLT
table_nm = 'STD_BD_GRD4_MLSFC_RSLT'.upper()

In [83]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769723445900345344 stmt_idx=1>

In [84]:
# 데이터 추가
# 7s
expdf = STD_BD_GRD4_MLSFC_RSLT
table_nm = 'STD_BD_GRD4_MLSFC_RSLT'.upper()
wbt.import_from_pandas(expdf, table_nm)

In [73]:
# # about 9.7s
# # 4등급 세분류 결과
# STD_BD_GRD4_MLSFC_RSLT
# df3.to_csv(os.path.join(df_fold, f'BD1-2_STD_BD_GRD4_MLSFC_RSLT.csv'), index=False)

# 분석2

In [85]:
carr.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호'],
      dtype='object')

In [86]:
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

## 등록 & 제원 정보 병합(말소 유지)

In [87]:
carr.shape

(1898763, 10)

In [88]:
csersr = carr.merge(srcr[['제원관리번호', '연료']], on='제원관리번호', how='left')
csersr.shape

(1898763, 11)

## 1\. 차량관리번호 기준 병합

In [89]:
csersr.shape

(1898763, 11)

In [90]:
# 58.3s
ersr = csersr.merge(hisr, on='차량관리번호', how='left')
ersr.shape

(4063734, 13)

## 2\. 차량말소YN 만 추출

In [91]:
errm = ersr[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)
errm.shape

(369841, 13)

## 3\. 변경일자 최신으로 차대번호 중복 제거

In [92]:
errm = errm.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

## 4\. 변경일자 2019.01.01 이상만 추출

In [93]:
errm = errm[errm['변경일자'] >= 20190101].reset_index(drop=True)

In [94]:
errm.shape

(367089, 13)

In [95]:
errm.columns

Index(['차대번호', '법정동코드', '차량말소YN_x', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차량말소YN_y', '변경일자'],
      dtype='object')

## 5\. 법정동 코드 정보 병합

In [96]:
errc = errm.merge(coder, on='법정동코드', how='left')

## 시도, 연도별 차량 대수

In [97]:
df['최초등록일자'].describe()

count    1515737.00
mean    20056364.03
std        66403.08
min     19650209.00
25%     20060622.00
50%     20081111.00
75%     20100510.00
max     20220614.00
Name: 최초등록일자, dtype: float64

In [98]:
dfm = df.copy()

In [101]:
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')

In [102]:
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

### 시군구명 앞쪽 지역명만 남기기(dfm)

In [103]:
# 시군구명 앞쪽 지역명만 남기기(dfm)
dfm['시군구_수정'] = dfm['시군구'].str.split(' ').str[0]

### 연료 지역별 등록차량대수

In [104]:
num_car_by_local = dfm.groupby(['연료', '시도', '시군구_수정', '최초등록일자_년', '최초등록일자_월'], as_index=False)['차대번호'].count()
num_car_by_local = num_car_by_local.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도', '최초등록일자_월':'월'})
num_car_by_local

,연료,시도,시군구_수정,연도,월,등록차량대수
0,CNG(압축천연가스),전라남도,신안군,2005,04,1
1,LPG(액화석유가스),강원도,강릉시,1990,03,1
2,LPG(액화석유가스),강원도,강릉시,1992,04,1
3,LPG(액화석유가스),강원도,강릉시,1993,05,1
4,LPG(액화석유가스),강원도,강릉시,1993,07,1
...,...,...,...,...,...,...
75670,휘발유,충청북도,충주시,2009,02,1
75671,휘발유,충청북도,충주시,2009,03,1
75672,휘발유,충청북도,충주시,2010,03,1
75673,휘발유,충청북도,충주시,2013,05,1


### 연료 지역별 차량대수

In [105]:
num_car_by_local2 = dfm.loc[dfm['최초등록일자'] < '20220601'].groupby(['연료', '시도', '시군구_수정'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'차량대수'})
num_car_by_local2

,연료,시도,시군구_수정,차량대수
0,CNG(압축천연가스),전라남도,신안군,1
1,LPG(액화석유가스),강원도,강릉시,67
2,LPG(액화석유가스),강원도,고성군,15
3,LPG(액화석유가스),강원도,동해시,184
4,LPG(액화석유가스),강원도,삼척시,21
...,...,...,...,...
814,NaN,전라북도,군산시,3
815,NaN,전라북도,익산시,1
816,NaN,충청북도,옥천군,1
817,NaN,충청북도,진천군,1


In [106]:
num_car_by_local2[['연도', '월']] = ['2022', '06']
num_car_by_local2

,연료,시도,시군구_수정,차량대수,연도,월
0,CNG(압축천연가스),전라남도,신안군,1,2022,06
1,LPG(액화석유가스),강원도,강릉시,67,2022,06
2,LPG(액화석유가스),강원도,고성군,15,2022,06
3,LPG(액화석유가스),강원도,동해시,184,2022,06
4,LPG(액화석유가스),강원도,삼척시,21,2022,06
...,...,...,...,...,...,...
814,NaN,전라북도,군산시,3,2022,06
815,NaN,전라북도,익산시,1,2022,06
816,NaN,충청북도,옥천군,1,2022,06
817,NaN,충청북도,진천군,1,2022,06


### 연료 지역별 말소 대수

In [107]:
errc['변경일자'].head()

0   20220628.00
1   20220628.00
2   20220628.00
3   20220628.00
4   20220628.00
Name: 변경일자, dtype: float64

In [108]:
errc['변경일자'] = errc['변경일자'].astype('str')

In [109]:
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

### 시군구명 앞쪽 지역명만 남기기(errc)

In [110]:
# 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]

In [111]:
grp_erase = errc.loc[errc['변경일자_년'] == '2022'].groupby(['변경일자_년', '변경일자_월', '연료', '시도', '시군구_수정'], as_index=False)['차대번호'].count()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도', '변경일자_월':'월'})
grp_erase = grp_erase.sort_values(['시도', '시군구_수정'])
grp_erase

,연도,월,연료,시도,시군구_수정,말소차량대수
0,2022,01,LPG(액화석유가스),강원도,강릉시,1
106,2022,01,경유,강원도,강릉시,26
353,2022,01,휘발유,강원도,강릉시,11
671,2022,02,경유,강원도,강릉시,38
917,2022,02,휘발유,강원도,강릉시,6
...,...,...,...,...,...,...
2256,2022,04,휘발유,충청북도,충주시,6
2609,2022,05,경유,충청북도,충주시,47
2829,2022,05,휘발유,충청북도,충주시,7
3168,2022,06,경유,충청북도,충주시,44


In [112]:
grp_erase[grp_erase['연도'] == '2022'].shape

(3378, 6)

In [113]:
num_car_by_local2['연도'].max()

'2022'

In [114]:
yr_list = []
mth_list = []
fuel_list = []
ctpv_list = []
sgg_list = []
yr = int(num_car_by_local2['연도'].max())
mth = int(num_car_by_local2['월'].max())
sl = num_car_by_local2.drop_duplicates(['시도', '시군구_수정']).reset_index(drop=True)
for ctpv, sgg in sl[['시도', '시군구_수정']].values:
    for fuel in sl['연료'].unique():
        for mth in range(mth - 4, mth + 1):
            mthm = f'{mth:0>2}'
            yr_list.append(str(yr))
            mth_list.append(mthm)
            fuel_list.append(fuel)
            ctpv_list.append(ctpv)
            sgg_list.append(sgg)
base = pd.DataFrame({'연도':yr_list, '월':mth_list, '연료':fuel_list, '시도':ctpv_list, '시군구_수정':sgg_list})

In [115]:
base

,연도,월,연료,시도,시군구_수정
0,2022,02,CNG(압축천연가스),전라남도,신안군
1,2022,03,CNG(압축천연가스),전라남도,신안군
2,2022,04,CNG(압축천연가스),전라남도,신안군
3,2022,05,CNG(압축천연가스),전라남도,신안군
4,2022,06,CNG(압축천연가스),전라남도,신안군
...,...,...,...,...,...
5175,2022,02,휘발유,인천광역시,남구
5176,2022,03,휘발유,인천광역시,남구
5177,2022,04,휘발유,인천광역시,남구
5178,2022,05,휘발유,인천광역시,남구


In [116]:
num_car_by_local2.head()

,연료,시도,시군구_수정,차량대수,연도,월
0,CNG(압축천연가스),전라남도,신안군,1,2022,06
1,LPG(액화석유가스),강원도,강릉시,67,2022,06
2,LPG(액화석유가스),강원도,고성군,15,2022,06
3,LPG(액화석유가스),강원도,동해시,184,2022,06
4,LPG(액화석유가스),강원도,삼척시,21,2022,06


In [117]:
base1 = base.merge(num_car_by_local2, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [118]:
base2 = base1.merge(num_car_by_local, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [119]:
base3 = base2.merge(grp_erase, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [120]:
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)
base3[['차량대수', '등록차량대수', '말소차량대수']].isnull().sum()

차량대수      0
등록차량대수    0
말소차량대수    0
dtype: int64

In [121]:
n = 5
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']
    # base3.loc[(i+1)*n - 3, '차량대수'] = base3.loc[(i+1)*n - 2, '차량대수'] + base3.loc[(i+1)*n - 2, '말소차량대수'] - base3.loc[(i+1)*n - 2, '등록차량대수']
    # base3.loc[(i+1)*n - 4, '차량대수'] = base3.loc[(i+1)*n - 3, '차량대수'] + base3.loc[(i+1)*n - 3, '말소차량대수'] - base3.loc[(i+1)*n - 3, '등록차량대수']
    # base3.loc[(i+1)*n - 5, '차량대수'] = base3.loc[(i+1)*n - 4, '차량대수'] + base3.loc[(i+1)*n - 4, '말소차량대수'] - base3.loc[(i+1)*n - 4, '등록차량대수']

In [122]:
today_date = datetime.today().strftime("%Y%m")
today_date

'202306'

In [123]:
base3['기준연월'] = today_date

In [125]:
base4 = base3[[
    '기준연월',
    '연도',
    '월', 
    '연료', 
    '시도', 
    '시군구_수정', 
    '차량대수',
]]

In [126]:
chc_col = {
    '기준연월':'CRTR_YM',
    '연도':'YR', 
    '월':'MM', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
    '차량대수':'VHCL_MKCNT', 
}

In [127]:
STD_BD_GRD4_RGN_CURSTT = base4.rename(columns=chc_col)
STD_BD_GRD4_RGN_CURSTT.columns

Index(['CRTR_YM', 'YR', 'MM', 'FUEL_CD', 'CTPV', 'SGG', 'VHCL_MKCNT'], dtype='object')

### [출력] STD_BD_GRD4_RGN_CURSTT

In [128]:
expdf = STD_BD_GRD4_RGN_CURSTT
table_nm = 'STD_BD_GRD4_RGN_CURSTT'.upper()

In [129]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769723445900345344 stmt_idx=3>

In [130]:
# 데이터 추가
# 5s
expdf = STD_BD_GRD4_RGN_CURSTT
table_nm = 'STD_BD_GRD4_RGN_CURSTT'.upper()
wbt.import_from_pandas(expdf, table_nm)

In [177]:
# # STD_BD_GRD4_RGN_CURSTT
# base3.to_excel(os.path.join(analysis_fold, 'BD1_통계_4등급차_시도_시군구_2022년_월별_차량현황.xlsx'), index=False)

# 분석3

In [131]:
dfm.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구', '최초등록일자_년',
       '최초등록일자_월', '최초등록일자_일', '시군구_수정'],
      dtype='object')

In [132]:
errc.columns

Index(['차대번호', '법정동코드', '차량말소YN_x', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차량말소YN_y', '변경일자', '시도', '시군구', '변경일자_년',
       '변경일자_월', '변경일자_일', '시군구_수정'],
      dtype='object')

## 연도 시도 차종별 차량 대수

### 현재 차량 대수

In [133]:
num_car_by_local1 = dfm.loc[dfm['최초등록일자'] < '20220601'].groupby(['연료', '시도', '차종'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수'})
num_car_by_local1

,연료,시도,차종,차량대수
0,CNG(압축천연가스),전라남도,화물,1
1,LPG(액화석유가스),강원도,승용,458
2,LPG(액화석유가스),강원도,승합,153
3,LPG(액화석유가스),강원도,화물,188
4,LPG(액화석유가스),경기도,승용,3342
...,...,...,...,...
194,NaN,서울특별시,승합,2
195,NaN,인천광역시,승용,3
196,NaN,전라남도,승용,1
197,NaN,전라북도,승용,4


In [134]:
num_car_by_local1['연도'] = '2022'
num_car_by_local1

,연료,시도,차종,차량대수,연도
0,CNG(압축천연가스),전라남도,화물,1,2022
1,LPG(액화석유가스),강원도,승용,458,2022
2,LPG(액화석유가스),강원도,승합,153,2022
3,LPG(액화석유가스),강원도,화물,188,2022
4,LPG(액화석유가스),경기도,승용,3342,2022
...,...,...,...,...,...
194,NaN,서울특별시,승합,2,2022
195,NaN,인천광역시,승용,3,2022
196,NaN,전라남도,승용,1,2022
197,NaN,전라북도,승용,4,2022


### 등록 차량 대수

In [135]:
num_car_by_local2 = dfm.groupby(['연료', '시도', '차종', '최초등록일자_년'], as_index=False)['차대번호'].count()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도'})
num_car_by_local2

,연료,시도,차종,연도,등록차량대수
0,CNG(압축천연가스),전라남도,화물,2005,1
1,LPG(액화석유가스),강원도,승용,1989,3
2,LPG(액화석유가스),강원도,승용,1990,4
3,LPG(액화석유가스),강원도,승용,1991,13
4,LPG(액화석유가스),강원도,승용,1992,5
...,...,...,...,...,...
2353,휘발유,충청북도,화물,1995,8
2354,휘발유,충청북도,화물,1996,5
2355,휘발유,충청북도,화물,1997,2
2356,휘발유,충청북도,화물,1998,2


### 말소 차량 대수

In [136]:
grp_erase = errc.groupby(['변경일자_년', '연료', '시도', '차종'], as_index=False)['차대번호'].count()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도'})
grp_erase = grp_erase.sort_values(['시도'])
grp_erase

,연도,연료,시도,차종,말소차량대수
0,2019,CNG(압축천연가스),강원도,화물,1
461,2021,휘발유,강원도,승용,856
396,2021,경유,강원도,화물,1160
395,2021,경유,강원도,특수,16
394,2021,경유,강원도,승합,411
...,...,...,...,...,...
288,2020,경유,충청북도,승합,343
287,2020,경유,충청북도,승용,1689
222,2020,LPG(액화석유가스),충청북도,화물,28
626,2022,경유,충청북도,특수,6


In [137]:
yr_list = []
fuel_list = []
ctpv_list = []
cd_list = []
yr = int(num_car_by_local2['연도'].max())
for ctpv in num_car_by_local1['시도'].unique():
    for fuel in sl['연료'].unique():
        for cd in ['승용', '승합', '화물', '특수']:
            for yrm in range(yr - 3, yr + 1):
                yr_list.append(str(yrm))
                fuel_list.append(fuel)
                ctpv_list.append(ctpv)
                cd_list.append(cd)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '시도':ctpv_list, '차종':cd_list})

In [138]:
base

,연도,연료,시도,차종
0,2019,CNG(압축천연가스),전라남도,승용
1,2020,CNG(압축천연가스),전라남도,승용
2,2021,CNG(압축천연가스),전라남도,승용
3,2022,CNG(압축천연가스),전라남도,승용
4,2019,CNG(압축천연가스),전라남도,승합
...,...,...,...,...
1083,2022,휘발유,충청북도,화물
1084,2019,휘발유,충청북도,특수
1085,2020,휘발유,충청북도,특수
1086,2021,휘발유,충청북도,특수


In [139]:
base1 = base.merge(num_car_by_local1, on=['연도', '연료', '시도', '차종'], how='left')
base1

,연도,연료,시도,차종,차량대수
0,2019,CNG(압축천연가스),전라남도,승용,NaN
1,2020,CNG(압축천연가스),전라남도,승용,NaN
2,2021,CNG(압축천연가스),전라남도,승용,NaN
3,2022,CNG(압축천연가스),전라남도,승용,NaN
4,2019,CNG(압축천연가스),전라남도,승합,NaN
...,...,...,...,...,...
1083,2022,휘발유,충청북도,화물,153.00
1084,2019,휘발유,충청북도,특수,NaN
1085,2020,휘발유,충청북도,특수,NaN
1086,2021,휘발유,충청북도,특수,NaN


In [140]:
base2 = base1.merge(num_car_by_local2, on=['연도', '연료', '시도', '차종'], how='left')
base2

,연도,연료,시도,차종,차량대수,등록차량대수
0,2019,CNG(압축천연가스),전라남도,승용,NaN,NaN
1,2020,CNG(압축천연가스),전라남도,승용,NaN,NaN
2,2021,CNG(압축천연가스),전라남도,승용,NaN,NaN
3,2022,CNG(압축천연가스),전라남도,승용,NaN,NaN
4,2019,CNG(압축천연가스),전라남도,승합,NaN,NaN
...,...,...,...,...,...,...
1083,2022,휘발유,충청북도,화물,153.00,NaN
1084,2019,휘발유,충청북도,특수,NaN,NaN
1085,2020,휘발유,충청북도,특수,NaN,NaN
1086,2021,휘발유,충청북도,특수,NaN,NaN


In [141]:
base3 = base2.merge(grp_erase, on=['연도', '연료', '시도', '차종'], how='left')
base3

,연도,연료,시도,차종,차량대수,등록차량대수,말소차량대수
0,2019,CNG(압축천연가스),전라남도,승용,NaN,NaN,NaN
1,2020,CNG(압축천연가스),전라남도,승용,NaN,NaN,NaN
2,2021,CNG(압축천연가스),전라남도,승용,NaN,NaN,NaN
3,2022,CNG(압축천연가스),전라남도,승용,NaN,NaN,NaN
4,2019,CNG(압축천연가스),전라남도,승합,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1083,2022,휘발유,충청북도,화물,153.00,NaN,3.00
1084,2019,휘발유,충청북도,특수,NaN,NaN,NaN
1085,2020,휘발유,충청북도,특수,NaN,NaN,NaN
1086,2021,휘발유,충청북도,특수,NaN,NaN,NaN


In [142]:
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)
base3[['차량대수', '등록차량대수', '말소차량대수']].isnull().sum()

차량대수      0
등록차량대수    0
말소차량대수    0
dtype: int64

In [143]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']
    # base3.loc[(i+1)*n - 3, '차량대수'] = base3.loc[(i+1)*n - 2, '차량대수'] + base3.loc[(i+1)*n - 2, '말소차량대수'] - base3.loc[(i+1)*n - 2, '등록차량대수']
    # base3.loc[(i+1)*n - 4, '차량대수'] = base3.loc[(i+1)*n - 3, '차량대수'] + base3.loc[(i+1)*n - 3, '말소차량대수'] - base3.loc[(i+1)*n - 3, '등록차량대수']
    # base3.loc[(i+1)*n - 5, '차량대수'] = base3.loc[(i+1)*n - 4, '차량대수'] + base3.loc[(i+1)*n - 4, '말소차량대수'] - base3.loc[(i+1)*n - 4, '등록차량대수']

In [144]:
today_date = datetime.today().strftime("%Y%m")
today_date

'202306'

In [145]:
base3['기준연월'] = today_date

In [146]:
base4 = base3[[
    '기준연월', 
    '연도', 
    '연료', 
    '시도', 
    '차종', 
    '차량대수', 
]]

In [147]:
chc_col = {
    '기준연월':'CRTR_YM',
    '연도':'YR', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '차량대수':'VHCL_MKCNT', 
}

In [148]:
STD_BD_GRD4_RGN_CURSTT_MOD = base4.rename(columns=chc_col)
STD_BD_GRD4_RGN_CURSTT_MOD.columns

Index(['CRTR_YM', 'YR', 'FUEL_CD', 'CTPV', '차종', 'VHCL_MKCNT'], dtype='object')

### [출력] STD_BD_GRD4_RGN_CURSTT_MOD

In [149]:
expdf = STD_BD_GRD4_RGN_CURSTT_MOD
table_nm = 'STD_BD_GRD4_RGN_CURSTT_MOD'.upper()

In [150]:
# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
wbt.execute(sql)

<ExaStatement session_id=1769723445900345344 stmt_idx=5>

In [151]:
# 데이터 추가
# 5s
expdf = STD_BD_GRD4_RGN_CURSTT_MOD
table_nm = 'STD_BD_GRD4_RGN_CURSTT_MOD'.upper()
wbt.import_from_pandas(expdf, table_nm)

In [ ]:
# # STD_BD_GRD4_RGN_CURSTT_MOD
# base3.to_excel(os.path.join(analysis_fold, 'BD1_통계_4등급차_시도_시군구_2022년_월별_차량현황.xlsx'), index=False)

# 코드 마지막